<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/tryouts/Slicing_and_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np 
import pandas as pd
import os
import pylab as pl
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path
from tqdm import tqdm
import shutil
from collections import Counter
from scipy.stats import entropy
import matplotlib.mlab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/MLTSA22/Final_Project

/content/drive/MyDrive/MLTSA22/Final_Project


In [4]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project

00000258_s002_t000.edf
all_tuh_eeg_seizure_links.txt
dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf
dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf
files_raw_channels_small_df.pkl
isip_tuh_eeg
seizures_types_v01.xlsx
temp


In [5]:
working_folder = Path('/content/drive/MyDrive/MLTSA22/Final_Project')

In [6]:
df = pd.read_pickle(
    working_folder.joinpath('files_raw_channels_small_df.pkl')
)

In [7]:
df.head()

,filename,category,configuration,index,patient_#,session,EEG FP1-REF,EEG FP2-REF,EEG F3-REF,EEG F4-REF,...,EEG T3-REF,EEG T4-REF,EEG T5-REF,EEG T6-REF,EEG FZ-REF,EEG CZ-REF,EEG PZ-REF,EEG A1-REF,EEG A2-REF,labels
file_name,,,,,,,,,,,,,,,,,,,,,
00010563_s002_t003,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[2.7726555877600295e-10, 1.3483732842699443e-0...","[-3.906051578001337e-10, 2.4807970640912983e-0...","[-1.9284389162122937e-10, 1.0694757868255472e-...","[-5.096480914428079e-11, 4.047438658809519e-06...",...,"[4.146017629996594e-09, -2.0413263864026096e-0...","[1.444056012902292e-09, 3.817044838007959e-07,...","[-2.910958452221951e-09, -3.250484807099009e-0...","[1.8423372397872572e-09, -7.1615811255855e-07,...","[-1.364304852525069e-10, 3.5260483039175664e-0...","[1.3531881762796245e-10, 6.983863806835304e-07...","[-2.3495119248263287e-10, -7.617383775562379e-...","[9.596059492877374e-10, -1.1888197912083615e-0...","[1.0519972902783143e-11, -2.1400490121479406e-...","[[0.0, 601.0, bckg]]"
00010563_s002_t004,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[-1.0278586830940462e-08, -1.7158036103551026e...","[2.7648617251111243e-09, -2.8490217427264524e-...","[3.288941204731503e-09, 4.617119704569293e-07,...","[3.399964489573073e-09, -5.345430061192608e-06...",...,"[8.658498750608017e-09, 1.7031133962228388e-07...","[1.780158122957924e-09, -2.152228442797806e-06...","[-4.916698865387427e-10, -2.9856703770832174e-...","[-6.258704822517387e-09, -1.8593219919890405e-...","[9.815260117792147e-10, -2.4137912796958633e-0...","[-2.357383015178091e-10, -5.355394184505534e-0...","[-1.0067458116184742e-09, -1.6493666048317114e...","[-4.7017904553916475e-09, -1.2061141044168896e...","[-1.1048988390260309e-08, -1.0048047870747276e...","[[0.0, 300.0, bckg]]"
00010563_s002_t005,train__01_tcp_ar__105__00010563__s002_2013_07_...,train,01_tcp_ar,105,00010563,s002_2013_07_17,"[3.7111311706420487e-09, -1.0474257222754713e-...","[7.219823200366963e-09, -3.1951117881894467e-0...","[2.1782993270312925e-09, -8.533492755460114e-0...","[-1.365985882998853e-09, 7.092433104769807e-06...",...,"[-8.948886072381209e-10, -7.939269490082801e-0...","[3.07454363042913e-09, 1.4140713109054442e-06,...","[6.674844103884688e-09, -4.037450817687943e-06...","[-3.61287734508141e-09, 6.247470115147276e-06,...","[-2.062432726832931e-09, 5.42573298081137e-07,...","[-8.346167589202069e-10, -3.2631357231385514e-...","[2.2304470728923623e-10, 3.7421245926520404e-0...","[1.4396886880297788e-09, -7.661145779436604e-0...","[6.327147966320168e-10, 1.1833053986962973e-05...","[[0.0, 601.0, bckg]]"
00010563_s003_t000,train__01_tcp_ar__105__00010563__s003_2013_07_...,train,01_tcp_ar,105,00010563,s003_2013_07_17,"[-4.837242318016071e-09, -1.9639659553544583e-...","[-1.7168631924658783e-09, 1.0420288689012175e-...","[-2.8357483900477646e-09, 1.3444340192373916e-...","[-4.3997702726288667e-10, -4.612748503245249e-...",...,"[-1.140736912419709e-09, 7.245232568610588e-07...","[2.6767669024911234e-09, 1.6102607124345142e-0...","[4.665380888551219e-09, 9.506710232528226e-07,...","[2.351093927809803e-09, 2.4800401575907515e-06...","[-1.0594241716063995e-10, -8.909057146788971e-...","[-4.790726036099662e-10, -1.4461463449301e-06,...","[4.02520870030525e-10, 1.1526165134603241e-08,...","[-5.771001920778108e-09, 3.624165205087055e-08...","[4.244714350965895e-09, -2.003986424867263e-06...","[[0.0, 300.0, bckg]]"
00010563_s003_t001,train__01_tcp_ar__105__00010563__s003_2013_07_...,train,01_tcp_ar,105,00010563,s003_2013_07_17,"[-5.683698910306182e-08, 2.654920991845486e-06...","[-3.337798055224077e-08, 2.8095137800880105e-0...","[3.7954605921399086e-08, 1.4489212326297262e-0...","[-1.2525189500901834e-08, 3.12269697219255e-06...",...,"[8.124202650339955e-08, 5.883211872988577e-07,...","[-4.9073445727403074e-08, 1.3883104676019777e-...","[1.9867273141848352e-08, 1.6805965603429214e-0...","[-6.6

In [8]:
df.columns

Index(['filename', 'category', 'configuration', 'index', 'patient_#',
       'session', 'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF', 'labels'],
      dtype='object')

In [9]:
# creating a new column, where there will be a list of labels
# the list will be the same for ALL the channels of a particular time series
df['seizure_type'] = np.NaN

In [10]:
df.columns

Index(['filename', 'category', 'configuration', 'index', 'patient_#',
       'session', 'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF', 'labels', 'seizure_type'],
      dtype='object')

Let's upload additional df with the types of the events/seizures. Because if there is a label, that DOESN'T mean that it's a SEIZURE

In [11]:
seizure_types = pd.read_excel(
    working_folder.joinpath('seizures_types_v01.xlsx')
)

In [12]:
seizure_types

,Class Code,Class No.,Event Name,Signs,Locality,Description
0,NaN,0,No Event,NaN,NaN,An unclassified event
1,SPSW,1,Spike/Sharp and Wave,Electrographic,All,"Spike and wave/complexes , sharp and wave/comp..."
2,GPED,2,Generalized Periodic Epileptiform Discharges,Electrographic,Generalized,Diffused periodic discharges
3,PLED,3,Periodic Lateralized Epileptiform Discharges,Electrographic,Hemispheric/Focal,Focal periodic discharges
4,EYBL,4,Eye blink,Clinical & Electrographic,Focal,"A specific type of sharp, high amplitude eye m..."
5,ARTF,5,Artifacts (All),Clinical & Electrographic,All,"Any non-brain activity electrical signal, such..."
6,BCKG,6,Background,Electrographic,NaN,Baseline/non-interesting events
7,SEIZ,7,Seizure,Clinical &| Electrographic,All,Common seizure class which can include all typ...
8,FNSZ,8,Focal Non-Specific Seizure,Electrographic,Hemispheric/Focal,Focal seizures which cannot be specified with ...
9,GNSZ,9,Generalized Non-Specific Seizure,Electrographic,Generalized,Generalized seizures which cannot be further c...


In [13]:
seiz_names = seizure_types['Class Code'].values

In [14]:
# based on the classification in the table, these are epileptic seizures
names = ['SEIZ', 'FNSZ', 'GNSZ', 'SPSZ', 'CPSZ', 
              'ABSZ', 'TNSZ', 'CNSZ', 'TCSZ', 'ATSZ', 
              'MYSZ']

In [15]:
seiz_names = []
for name in names:
  seiz_names.append(name.lower())

In [16]:
# in the .tse files, the names are in lower case
seiz_names

['seiz',
 'fnsz',
 'gnsz',
 'spsz',
 'cpsz',
 'absz',
 'tnsz',
 'cnsz',
 'tcsz',
 'atsz',
 'mysz']

Column 'labels' have np.array files - matrices.

In [17]:
tryout = df.at['00010563_s002_t003', 'EEG P3-REF']

In [18]:
label = df.at['00010563_s002_t003', 'labels']

In [19]:
label.shape

(1, 3)

## def labeling

In the df, in the column 'labels' there are data from .tse file containing start, end and type of seizure. .tse file could contain several labels. So, the 'labels' column is np.array, which contain three columns and MIGHT contain more than one row. In the function **labeling** in the first loop, we account for possible several rows. Then we calculate the start and end of the seizure in the number of samples according to our sampling frequency. In the next loop, we are comparing the type of seizure that is contained in a particular row of labels array and compare to our target types which are seizures (the list is above).

There could be several ways how a seizure could be captured inside the window. 1 - if the seizure starts before the window and ends before the end of the window. 2 - if the seizure is within the window. 3 - if the seizure starts within the window and ends outside of the window. 4 - if seizure event is greater than window itself and so seizure event contains the window. In all of these cases we assign label 1. If there is no seizure in the window or the event is not seizure - we assing 0.

In [23]:
def labeling(label):
  #label - array object from the column labels containing start, end and type of seizure

  sampl_freq = 200 # Fixed value after downsampling
  labels = [] # list for labels

    for i in seiz_names:
      if label[row][2] == i:    
        #conditions if there is a seizure type in the .tse file     
        if (start <= counter and
            end > counter and
            end <= counter+window_size):
            # 1 seizure starts before window and ends within
          labels.append(1)
        elif (start >= counter and
              end <= counter+window_size):
          # 2 seizure within window
          labels.append(1)
        elif (start >= counter and
            start < counter+window_size and
            end > counter+window_size):
          # 3 seizure starts before the end of window
          labels.append(1)
        elif (start <= counter and
              end >= counter+window_size):
          # 4 window is within seizure
          labels.append(1)
        else:
          # no seizure within the window
          labels.append(0)
      
        #if the type in the .tse file is not a seizure
      else: 
        labels.append(0)
      break

  return labels

## def slicing

In this function, we assign the window size and step size and cut our data into chuncks. For each of the chuncks, we label it and add it to the list (see the slack for the discussion on this one), standardize, and add to the new list which corresponds to particualr channel and time series and contains all the chuncks.

In [22]:
def slicing(timeseries, label): 
  # timeseries - list of values from each cell 
  #label - array object from the column labels containing start, end and type of seizure

  # parameters for slicing 
  window_size = 10000
  step_size = 500
  sliced = [] # list for timeseries chunks
  counter = 0

  labels = []

  stan_timeseries = (timeseries - np.mean(timeseries)) / np.std(timeseries) # standardizing

  while counter < len(stan_timeseries):
    series = stan_timeseries[counter:counter+window_size]
    sliced.append(series)

    #labeling for slices
    labels.append(labeling(label))

    counter += window_size
    counter += step_size

  return sliced, labels

In [40]:
columns = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF',
       'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF',
       'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF',
       'EEG T5-REF', 'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF',
       'EEG A1-REF', 'EEG A2-REF']

In [41]:
for i in columns:
  data = df.at['00010563_s002_t003', i]
  label = df.at['00010563_s002_t003', 'labels']
  slices, labels = slicing(data, label)
  



In [ ]:
tryout = df.at['00010563_s002_t003', 'EEG P3-REF']
label = df.at['00010563_s002_t003', 'labels']

In [25]:
slices, labels = slicing(tryout, label)

In [42]:
slices

[array([ 4.32387807e-04, -3.96642387e-03,  1.63228345e-02, ...,
        -3.97619458e-01, -4.12636619e-01, -4.39963280e-01]),
 array([0.04995118, 0.10929153, 0.17613413, ..., 0.399677  , 0.3743713 ,
        0.34943389]),
 array([-0.09597912, -0.0847086 , -0.10756585, ...,  0.03810767,
        -0.04295782, -0.08950665]),
 array([ 0.20611755,  0.13366414,  0.08826121, ..., -0.12404045,
        -0.11735786, -0.10622137]),
 array([ 0.30337901,  0.3471552 ,  0.3540662 , ..., -0.09746155,
        -0.08776749, -0.09180357]),
 array([-0.19627029, -0.23067434, -0.20511026, ...,  0.02590214,
         0.11659252,  0.18376416]),
 array([-0.05419005, -0.09803493, -0.14949959, ..., -2.90291876,
        -2.7316967 , -2.87324102]),
 array([ 1.67122192,  1.59924668,  1.54208952, ..., -0.24308973,
        -0.21339914, -0.14671416]),
 array([0.21578183, 0.22268621, 0.17882814, ..., 0.243952  , 0.1934714 ,
        0.18124102]),
 array([-0.26042462, -0.29684349, -0.30399198, ..., -0.07089454,
        -0.065

In [33]:
len(slices[0])

10000

In [35]:
len(labels)

12

In [43]:
slice = slices[0]

In [44]:
slice

array([ 4.32387807e-04, -3.96642387e-03,  1.63228345e-02, ...,
       -3.97619458e-01, -4.12636619e-01, -4.39963280e-01])

Feature extraction

In [51]:
def feature_extraction(slice):
  # takes as an input one window in parallel with the windows from other channels

  ##time domain
  # mean
  mean = np.mean(slice)
  # variance
  var = np.var(slice)
  # maximum
  max = np.max(slice)
  # minimum
  min = np.min(slice)
  # standard deviation
  std = np.std(slice)
  # line length   
  line_length = 0
  for i in range(0, len(slice)):
    if i == 0:
      line_length += abs(slice[i])
    else:
      line_length += abs(slice[i] - slice[i-1])  
  # entropy
  entr = entropy(slice) # the result is infinite

  ##frequency domain
  psd = matplotlib.mlab.psd(slice)
  power = psd[0].tolist()
  freqs = psd[1].tolist()

  low_freq = 0
  theta = 0
  alpha = 0
  beta = 0
  gamma = 0
  for i in range(0, len(psd[0])):
    # low frequency power
    if freqs[i] <= 0.04:
      low_freq += power[i]   
    #theta total power 4 - 8 Hz
    if 0.04 < freqs[i] <= 0.08:
      theta += power[i]
    # alpha total power
    if 0.08 < freqs[i] <= 0.12:
      alpha += power[i]
    # beta total power
    if 0.13 < freqs[i] <= 0.3:
      beta += power[i]
    # gamma total power
    if 0.3 < freqs[i] <= 1:
      gamma += power[i]
    
  # peak frequency
  peak_freq = freqs[power.index(max(power))]

   

  # magnitude of psd
  mag_psd = matplotlib.mlab.magnitude_spectrum(slice)[0]# first array is magnitude, the second - frequencies
  # angle of spectrum
  ang_psd = matplotlib.mlab.angle_spectrum(slice)[0]
  # phase of spectrum
  ph_psd = matplotlib.mlab.phase_spectrum(slice)[0]



  ##mixed domain

  return mean, var, max, min, line_length, entr


In [85]:
psd = matplotlib.mlab.psd(slice)
  

In [95]:
np.where(psd[0] == psd[0].max())

(array([1]),)

In [114]:
freqs[power.index(max(power))]

0.0078125

In [81]:
import pyhrv
import pyhrv.frequency_domain as fd

In [83]:
result = fd.welch_psd(np.array(slice))

ValueError: ignored

In [ ]:
result

In [80]:
pip install pyhrv

     |████████████████████████████████| 3.2 MB 5.1 MB/s 
     |████████████████████████████████| 230 kB 48.0 MB/s 
     |████████████████████████████████| 1.5 MB 1.3 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
  Created wheel for biosppy: filename=biosppy-0.8.0-py2.py3-none-any.whl size=100176 sha256=244bfa33b5c4673ba93c5995cfd99ef5f3f6923c1fce863ebf79ddd4ce5cb00d
  Stored in directory: /root/.cache/pip/wheels/32/17/9d/77adbf0db713546eaa1c7a1480b2dce91fa724a133115dc20c
  Created wheel for spectrum: filename=spectrum-0.8.1-cp37-cp37m-linux_x86_64.whl size=235154 sha256=e28740f449b374e7013bbc3c77e28cc876af20ee4644da84fd6f9590c552cde3
  Stored in directory: /root/.cache/pip/wheels/79/db/9c/92fa684ca088447807d08672e7609b48102c6161ac9c7e3c62
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64232 sha256=cf7b704bc3c481de16150372e5058169e72a29f7b755717a76537c26d87ffcc5
  Stored in directory: /root/.cache/pip/wheels/82/ab/83/fdfc4017ea44a585b6754752cc